In [ ]:
import eikon as ek
import pandas as pd
from tqdm import tqdm

import json
import os
import sys
from dotenv import load_dotenv

load_dotenv()
ek.set_app_key(os.getenv('EIKON_API_KEY'))
repo_path = os.getenv('REPO_PATH')

sys.path.insert(0, repo_path + r'src_HF')
from utils.main_utils import *
from utils.gather_utils import *


### Gather data with EIKON

In [ ]:
headline_topic = 'CWP'

headline_df = pd.read_csv(repo_path + rf'data\raw_news_headlines\EIKON_{headline_topic}_NEWS.csv')

previous_stories = load_previous_stories(headline_topic)

existing_ids = previous_stories.keys()

# get all story ids not in the existing_ids series
storie_ids = headline_df[~headline_df['storyId'].isin(existing_ids)]['storyId']

print(f'Number of stories alredy downloaded/failed: ({len(existing_ids)}/{len(headline_df)})')

# download the stories
new_dict = extract_stories(storie_ids)


### Update existing dictionary

In [ ]:
# update the text_dict with the new_dict
previous_stories.update(new_dict)

# save the updated text_dict
with open(repo_path + rf'data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.json', 'w') as f:
    json.dump(previous_stories, f, indent=2)


### Error retry

In [ ]:
previous_stories = load_previous_stories(headline_topic)

error_ids = [k for k, v in previous_stories.items() if v == 'error']

print(f'Number of stories with errors: {len(error_ids)}')

# download the stories
new_dict = extract_stories(error_ids)


In [ ]:
# update the text_dict with the new_dict
previous_stories.update(new_dict)

# save the updated text_dict
with open(repo_path + rf'data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.json', 'w') as f:
    json.dump(previous_stories, f, indent=2)